In [1]:
import os
import math
import numpy as np
import pandas as pd
from datetime import datetime, date, timedelta, time
os.environ['TZ'] ='America/New_York'

#import yfinance as yf

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

from ib_async import *
util.startLoop()

In [2]:
ib = IB()
ib.connect(port=7496, clientId=0)

<IB connected to 127.0.0.1:7496 clientId=0>

In [4]:
def simple_data_req( contract, end_date , barsize ="5 mins", data_clean = True):

   
    df_acc=pd.DataFrame()
    df_hist=pd.DataFrame()

    for i in range(1):
        print(f'fetching data for {i}')
        end_date = end_date - timedelta(i)
        bars = ib.reqHistoricalData(contract= contract, endDateTime=end_date, barSizeSetting= barsize, durationStr= "230 D", whatToShow="TRADES", useRTH=True)
        df = util.df(bars)
        
        df_acc = pd.concat([df, df_acc], ignore_index=True)
        print(f'head date {df_acc.head(1)['date']}')
        print(f'tail date {df_acc.tail(1)['date']}')

    df_acc['Date_Only'] = df_acc['date'].dt.date
    
    if data_clean == True:
        df_hist = df_acc[(df_acc['date'].dt.time >= time(9, 30)) & (df_acc['date'].dt.time < time(16, 00))]
    else:
        df_hist = df_acc

    #df_hist.drop_duplicates(inplace=True)
    
    return df_hist

In [3]:
contract = Contract(symbol='TQQQ', secType='STK', exchange='SMART', currency='USD')
ib.qualifyContracts(contract)

[Contract(secType='STK', conId=72539702, symbol='TQQQ', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='TQQQ', tradingClass='NMS')]

In [5]:
now =datetime.now()
inferencing_date = now.date() - timedelta(1)

In [6]:
df_TQQQ = simple_data_req(contract, inferencing_date,)

fetching data for 0
head date 0   2024-05-20 09:30:00-04:00
Name: date, dtype: datetime64[ns, US/Eastern]
tail date 17831   2025-04-21 15:55:00-04:00
Name: date, dtype: datetime64[ns, US/Eastern]


In [13]:
df_TQQQ['diff_close_open'] = df_TQQQ['open'] - df_TQQQ['close'].shift(1)

In [15]:
df_TQQQ.iloc[78]

date               2024-05-21 09:30:00-04:00
open                                   63.44
high                                   63.68
low                                    63.35
close                                  63.66
volume                             1598649.0
average                               63.482
barCount                                4912
Date_Only                         2024-05-21
diff_close_open                        -0.82
Name: 78, dtype: object

In [16]:
def distance(data, acc_volume):
    buy = data.close -data.low
    sell = data.high - data.close
    dist_BS = (buy - sell)*data.volume/acc_volume
    #dist_BS = (buy - sell)
    return dist_BS

In [17]:
def mid_price(data):
    mid = (data.high+data.low)/2
    return mid

In [19]:
df = df_TQQQ.copy()

In [ ]:
acc_distance = 0
acc_volume = 0
mid = []

def feature_creation(df, initial_acc, barsize=5 ):
    
    for n in range(230):
        print(f'period: {n}')
        flag = ''
        Period = (30/barsize + 6*60/barsize)*n
        
        for i in range(initial_acc):
            acc_volume += df.iloc[i+Period].volume
        
    # need to find acc_volume first        
        for i in range(initial_acc):
            dist_BS = distance(df.iloc[i+Period], acc_volume)
            acc_distance += dist_BS
            mid.append(mid_price(df.iloc[i+Period]))
        
        mid_price_adj = mid[2]- mid[0]
        

    
        if flag =='BUY':
    
            flag_below='False'
            flag_above ='False'
            
            trade_width = (df_GOOG.iloc[0+Test].high -df_GOOG.iloc[0+Test].low)*2
            win_amount = 0
            
            for i in range(50):
                if df_GOOG.iloc[i+Test].low < df_GOOG.iloc[0+Test].high and  i>2:
                    print(i)
                    print(f'close: {df_GOOG.iloc[i+Test].close}')
                    print(f' bought at {df_GOOG.iloc[0+Test].high}')
                    flag_below ='True'
                elif (df_GOOG.iloc[i+Test].close -df_GOOG.iloc[i+Test].open)/(df_GOOG.iloc[i+Test].high -df_GOOG.iloc[i+Test].low)>0.8 :
                    print(f' flag_high bought at {df_GOOG.iloc[i+Test].close}')
                    bought_at =df_GOOG.iloc[i+Test].close
                    flag_above ='True'
                else:
                    break
                    
                                                
                if flag_below ==True and  df_GOOG.iloc[i+Test].high > df_GOOG.iloc[0+Test].high:
                    print('TRADE!')
    
                if flag_below == 'True':
                    if df_GOOG.iloc[i+Test].high > (df_GOOG.iloc[0+Test].high+trade_width):
                        win_amount += trade_width
                        print(f'close and won {win_amount}')
                        break
                    if df_GOOG.iloc[i+Test].low < df_GOOG.iloc[0+Test].low:
                        win_amount -= trade_width/2
                        print(f'close and won $ {win_amount}')
                        break
    
                if flag_above =='True':
                    
                    if df_GOOG.iloc[i+Test].high > (df_GOOG.iloc[i+Test].close+trade_width*2):
                        win_amount += trade_width
                        print(f'close and won {win_amount}')
                        break
                    else:
                        win_amount += df_GOOG.iloc[77+Test].high -bought_at
                    


In [ ]:
period = 1
df_GOOG.iloc[0+78*period:40+78*period].close.plot()

In [ ]:
df_GOOG.iloc[78].high

In [ ]:
df_GOOG.iloc[78].close

In [ ]:
df_GOOG.iloc[8]

In [ ]:
df_GOOG.iloc[0:80].close.plot()

In [ ]:
df_GOOG.iloc[0:80].close.plot()

In [ ]:
for index, row in df_test.iterrows():
    dist_BS = distance(row)
    if dist_BS >0:
        print(index)

In [ ]:
df_GOOG.iloc[1]

In [ ]:
df_high_second = df_high = df_GOOG.iloc[1]['high']
df_low_second = df_GOOG.iloc[1]['low']
df_mid_second = (df_high_second+df_low_second)/2

In [ ]:
df_mid_second

In [ ]:
df_test = df_GOOG[:100]

In [ ]:
df_test['close'].plot()

In [ ]:
for index, row in df_test.iterrows():
    if row.high < df_mid and index > 2:
        print(index)

    if row.low > df_low:
        print(f'low : {index}')
    if row.high < df_mid:
        print(f'high : {index}')
        

In [ ]:
# clear and close it